In [21]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
TRAIN_DATASET_PATH = '/content/drive/My Drive/ds1/train.csv'
TEST_DATASET_PATH = '/content/drive/My Drive/ds1/test.csv' 

In [50]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()
T = test_df.values
train_df.columns
target = 'choose'
X = train_df.drop(columns = target)
y = train_df[target]

X = X.values
y = y.values

In [27]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [28]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [29]:
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1 # коэффициент неопределенности Джини
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [30]:
def entropy(labels):
  classes = {}
  for label in labels:
      if label not in classes:
          classes[label] = 0
      classes[label] += 1

  chaos = 0
  for label in classes:
    p =  classes[label] / len(labels)
    if p != 0:
      chaos -= p * np.log2(p)
  return chaos

In [31]:
# Расчет качества

def quality(left_labels, right_labels, current_criterion, criterion='gini'):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

    if criterion == 'entropy':
        return current_criterion - p * entropy(left_labels) - (1 - p) * entropy(right_labels)
    
    return current_criterion - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [32]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [33]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, min_leaf, criterion='gini'):

    if criterion == 'entropy':
      current_criterion = entropy(labels)
    
    current_criterion = gini(labels)
      
    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            if criterion == 'entropy':
              current_quality = quality(true_labels, false_labels, current_criterion, criterion='entropy')
              
            current_quality = quality(true_labels, false_labels, current_criterion, criterion='gini')
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [34]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, min_leaf=5, max_depth=None, depth=0, max_leaves=1000, criterion='gini'):
    if criterion == 'entropy':
      quality, t, index = find_best_split(data, labels, min_leaf, criterion='entropy')
    
    quality, t, index = find_best_split(data, labels, min_leaf, criterion='gini')

    global counter_node
    global counter_leaf

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        counter_leaf += 1
        return Leaf(data, labels)

    if (max_depth is not None) and (depth >= max_depth):
        counter_leaf += 1
        return Leaf(data, labels)

    if max_leaves is not None:
        if counter_leaf <= max_leaves or counter_node <= (max_leaves/2):
              true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
              counter_node += 1
              depth += 1

    # Рекурсивно строим два поддерева
              true_branch = build_tree(true_data, true_labels, 
                                     min_leaf, max_depth, depth, (max_leaves-counter_leaf))
              false_branch = build_tree(false_data, false_labels, 
                                      min_leaf, max_depth, depth, (max_leaves-counter_leaf))
        elif (counter_leaf > max_leaves) or ((counter_node + 1) > (max_leaves / 2)):
              counter_leaf += 1
              return Leaf(data, labels)
        
    else:
        true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
        counter_node += 1
        depth += 1 

        # Рекурсивно строим два поддерева
        true_branch = build_tree(true_data, true_labels, 
                                 min_leaf, max_depth, depth, max_leaves)
        false_branch = build_tree(false_data, false_labels, 
                                  min_leaf, max_depth, depth, max_leaves)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [35]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [36]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [64]:
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [65]:
counter_leaf = 0 
counter_node = 0
my_tree = build_tree(X_train, y_train, min_leaf=5, max_depth=10)

In [70]:
train_answers = predict(X_train, my_tree)
test_answers = predict(x_test, my_tree)
answers = predict(T, my_tree)

In [57]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [68]:
train_accuracy = accuracy_metric(y_train, train_answers)
test_accuracy = accuracy_metric(y_test, test_answers)
print(test_accuracy)

88.7


In [59]:
submit = pd.read_csv('/content/drive/My Drive/ds1/submission_example.csv')
submit.head()

,Id,choose
0,10000,0.5
1,10001,0.5
2,10002,0.5
3,10003,0.5
4,10004,0.5


In [61]:
submit['choose'] = answers
submit.head()

,Id,choose
0,10000,0
1,10001,0
2,10002,0
3,10003,0
4,10004,0


In [63]:
submit.to_csv('Bazhan_submit_class2.csv', index=False)

# Новый раздел